# load library

In [1]:
import pandas as pd
from pathlib import Path
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

# load dataset

In [29]:
df = pd.read_csv('../csvs/current_dataset.csv', encoding='utf-8-sig')
df.head()

,video,yt_id,duration,title,singer,audio,lyric,year,submatch,match,...,mfcc_10,mfcc_11,mfcc_12,contrast_0,contrast_1,contrast_2,contrast_3,contrast_4,contrast_5,contrast_6
0,BBIBBI (삐삐).wav,BZqfL_2CzKg,3:15,BBIBBI,IU,Q1,Q2,2018,0,1,...,-5.839810,1.736630,-4.027808,19.489751,16.817562,19.707883,18.932899,19.135302,18.334705,48.933727
1,Twenty-three (스물셋).wav,GXNHaLIhyMY,3:15,스물셋,IU,Q1,Q2,2015,0,1,...,-6.213743,5.054786,-1.629375,19.753127,14.671149,17.610113,17.852948,17.890963,16.211872,48.490606
2,Jam Jam (잼잼).wav,KWjDSRdIFgc,3:39,잼잼,IU,Q1,Q2,2017,0,1,...,-1.621268,0.532002,0.867949,20.835910,16.579150,18.995342,17.872959,17.487224,16.984500,48.026803
3,다비치 (DAVICHI) - 너에게 못했던 내 마지막 말은 (Unspoken Wor...,inxtzi3d_0I,3:25,너에게 못했던 내 마지막 말은,다비치,Q1,Q3,2019,0,1,...,-12.059926,0.593395,-4.183328,19.438179,17.152836,22.308247,22.202007,21.834402,22.584914,47.920954
4,시간아 멈춰라 (시간아 멈춰라).wav,i-SBnhaZSSU,4:29,시간아 멈춰라,다비치,Q1,Q2,2010,0,1,...,-8.969615,3.669938,-4.159402,22.995046,17.665243,19.659806,20.755871,20.048621,19.850872,48.472762


In [30]:
df['submatch'].value_counts()

submatch
2    45
0    31
3    27
1    15
Name: count, dtype: int64

In [31]:
emo = pd.read_csv('../csvs/audio_emotion_results.csv', encoding='utf-8-sig')
emo.head()

,file,valence,arousal,quadrant
0,[Lyrics Video] BOL4(볼빨간사춘기) - Blue.wav,0.547980,0.494141,Q4
1,"4.처음엔 사랑이란게 - Love, At First.wav",0.442252,0.434688,Q3
2,Twenty-three (스물셋).wav,0.761262,0.642141,Q1
3,U & I (U & I).wav,1.010887,0.952694,Q1
4,버스커 버스커(Busker Busker) - 벚꽃 엔딩 [가사_Lyrics].wav,0.587622,0.558414,Q1


In [32]:
emo.rename(columns={'file':'video'}, inplace=True)
emo.head()

,video,valence,arousal,quadrant
0,[Lyrics Video] BOL4(볼빨간사춘기) - Blue.wav,0.547980,0.494141,Q4
1,"4.처음엔 사랑이란게 - Love, At First.wav",0.442252,0.434688,Q3
2,Twenty-three (스물셋).wav,0.761262,0.642141,Q1
3,U & I (U & I).wav,1.010887,0.952694,Q1
4,버스커 버스커(Busker Busker) - 벚꽃 엔딩 [가사_Lyrics].wav,0.587622,0.558414,Q1


In [40]:
A_updated = df.merge(emo, on='video', how='left')
A_updated.head()

,video,yt_id,duration,title,singer,audio,lyric,year,submatch,match,...,contrast_0,contrast_1,contrast_2,contrast_3,contrast_4,contrast_5,contrast_6,valence,arousal,quadrant
0,BBIBBI (삐삐).wav,BZqfL_2CzKg,3:15,BBIBBI,IU,Q1,Q2,2018,0,1,...,19.489751,16.817562,19.707883,18.932899,19.135302,18.334705,48.933727,0.703667,0.642463,Q1
1,Twenty-three (스물셋).wav,GXNHaLIhyMY,3:15,스물셋,IU,Q1,Q2,2015,0,1,...,19.753127,14.671149,17.610113,17.852948,17.890963,16.211872,48.490606,0.761262,0.642141,Q1
2,Jam Jam (잼잼).wav,KWjDSRdIFgc,3:39,잼잼,IU,Q1,Q2,2017,0,1,...,20.835910,16.579150,18.995342,17.872959,17.487224,16.984500,48.026803,0.385379,0.390117,Q3
3,다비치 (DAVICHI) - 너에게 못했던 내 마지막 말은 (Unspoken Wor...,inxtzi3d_0I,3:25,너에게 못했던 내 마지막 말은,다비치,Q1,Q3,2019,0,1,...,19.438179,17.152836,22.308247,22.202007,21.834402,22.584914,47.920954,0.805343,0.747630,Q1
4,시간아 멈춰라 (시간아 멈춰라).wav,i-SBnhaZSSU,4:29,시간아 멈춰라,다비치,Q1,Q2,2010,0,1,...,22.995046,17.665243,19.659806,20.755871,20.048621,19.850872,48.472762,0.652898,0.612346,Q1


In [41]:
A_updated.drop(columns=['audio'], inplace=True)
A_updated.head()

,video,yt_id,duration,title,singer,lyric,year,submatch,match,tempo_transition_complexity,...,contrast_0,contrast_1,contrast_2,contrast_3,contrast_4,contrast_5,contrast_6,valence,arousal,quadrant
0,BBIBBI (삐삐).wav,BZqfL_2CzKg,3:15,BBIBBI,IU,Q2,2018,0,1,0.072606,...,19.489751,16.817562,19.707883,18.932899,19.135302,18.334705,48.933727,0.703667,0.642463,Q1
1,Twenty-three (스물셋).wav,GXNHaLIhyMY,3:15,스물셋,IU,Q2,2015,0,1,0.069901,...,19.753127,14.671149,17.610113,17.852948,17.890963,16.211872,48.490606,0.761262,0.642141,Q1
2,Jam Jam (잼잼).wav,KWjDSRdIFgc,3:39,잼잼,IU,Q2,2017,0,1,0.150170,...,20.835910,16.579150,18.995342,17.872959,17.487224,16.984500,48.026803,0.385379,0.390117,Q3
3,다비치 (DAVICHI) - 너에게 못했던 내 마지막 말은 (Unspoken Wor...,inxtzi3d_0I,3:25,너에게 못했던 내 마지막 말은,다비치,Q3,2019,0,1,0.124787,...,19.438179,17.152836,22.308247,22.202007,21.834402,22.584914,47.920954,0.805343,0.747630,Q1
4,시간아 멈춰라 (시간아 멈춰라).wav,i-SBnhaZSSU,4:29,시간아 멈춰라,다비치,Q2,2010,0,1,0.199776,...,22.995046,17.665243,19.659806,20.755871,20.048621,19.850872,48.472762,0.652898,0.612346,Q1


In [42]:
A_updated.rename(columns={'quadrant':'audio'}, inplace=True)
A_updated.head()

,video,yt_id,duration,title,singer,lyric,year,submatch,match,tempo_transition_complexity,...,contrast_0,contrast_1,contrast_2,contrast_3,contrast_4,contrast_5,contrast_6,valence,arousal,audio
0,BBIBBI (삐삐).wav,BZqfL_2CzKg,3:15,BBIBBI,IU,Q2,2018,0,1,0.072606,...,19.489751,16.817562,19.707883,18.932899,19.135302,18.334705,48.933727,0.703667,0.642463,Q1
1,Twenty-three (스물셋).wav,GXNHaLIhyMY,3:15,스물셋,IU,Q2,2015,0,1,0.069901,...,19.753127,14.671149,17.610113,17.852948,17.890963,16.211872,48.490606,0.761262,0.642141,Q1
2,Jam Jam (잼잼).wav,KWjDSRdIFgc,3:39,잼잼,IU,Q2,2017,0,1,0.150170,...,20.835910,16.579150,18.995342,17.872959,17.487224,16.984500,48.026803,0.385379,0.390117,Q3
3,다비치 (DAVICHI) - 너에게 못했던 내 마지막 말은 (Unspoken Wor...,inxtzi3d_0I,3:25,너에게 못했던 내 마지막 말은,다비치,Q3,2019,0,1,0.124787,...,19.438179,17.152836,22.308247,22.202007,21.834402,22.584914,47.920954,0.805343,0.747630,Q1
4,시간아 멈춰라 (시간아 멈춰라).wav,i-SBnhaZSSU,4:29,시간아 멈춰라,다비치,Q2,2010,0,1,0.199776,...,22.995046,17.665243,19.659806,20.755871,20.048621,19.850872,48.472762,0.652898,0.612346,Q1


In [44]:
import pandas as pd


# 조건 1: audio ∈ {1,4} & lyric ∈ {2,3} → submatch = 0, match = 1
cond1 = A_updated['audio'].isin([1, 4]) & A_updated['lyric'].isin([2, 3])
A_updated.loc[cond1, 'submatch'] = 0
A_updated.loc[cond1, 'match'] = 1

# 조건 2: audio ∈ {2,3} & lyric ∈ {1,4} → submatch = 1, match = 1
cond2 = A_updated['audio'].isin([2, 3]) & A_updated['lyric'].isin([1, 4])
A_updated.loc[cond2, 'submatch'] = 1
A_updated.loc[cond2, 'match'] = 1

# 조건 3: audio ∈ {1,4} & lyric ∈ {1,4} → submatch = 2, match = 0
cond3 = A_updated['audio'].isin([1, 4]) & A_updated['lyric'].isin([1, 4])
A_updated.loc[cond3, 'submatch'] = 2
A_updated.loc[cond3, 'match'] = 0

# 조건 4: audio ∈ {2,3} & lyric ∈ {2,3} → submatch = 3, match = 0
cond4 = A_updated['audio'].isin([2, 3]) & A_updated['lyric'].isin([2, 3])
A_updated.loc[cond4, 'submatch'] = 3
A_updated.loc[cond4, 'match'] = 0


In [45]:
A_updated['submatch'].value_counts()

submatch
2    45
0    31
3    27
1    15
Name: count, dtype: int64

In [46]:
A_updated['match'].value_counts()

match
0    72
1    46
Name: count, dtype: int64

In [22]:
A_updated.to_csv('new_dataset.csv', encoding='utf-8-sig', index=False)
A_updated.head()

,video,yt_id,duration,title,singer,lyric,year,submatch,match,tempo_transition_complexity,...,contrast_0,contrast_1,contrast_2,contrast_3,contrast_4,contrast_5,contrast_6,valence,arousal,audio
0,BBIBBI (삐삐).wav,BZqfL_2CzKg,3:15,BBIBBI,IU,Q2,2018,0,1,0.072606,...,19.489751,16.817562,19.707883,18.932899,19.135302,18.334705,48.933727,0.703667,0.642463,Q1
1,Twenty-three (스물셋).wav,GXNHaLIhyMY,3:15,스물셋,IU,Q2,2015,0,1,0.069901,...,19.753127,14.671149,17.610113,17.852948,17.890963,16.211872,48.490606,0.761262,0.642141,Q1
2,Jam Jam (잼잼).wav,KWjDSRdIFgc,3:39,잼잼,IU,Q2,2017,0,1,0.150170,...,20.835910,16.579150,18.995342,17.872959,17.487224,16.984500,48.026803,0.385379,0.390117,Q3
3,다비치 (DAVICHI) - 너에게 못했던 내 마지막 말은 (Unspoken Wor...,inxtzi3d_0I,3:25,너에게 못했던 내 마지막 말은,다비치,Q3,2019,0,1,0.124787,...,19.438179,17.152836,22.308247,22.202007,21.834402,22.584914,47.920954,0.805343,0.747630,Q1
4,시간아 멈춰라 (시간아 멈춰라).wav,i-SBnhaZSSU,4:29,시간아 멈춰라,다비치,Q2,2010,0,1,0.199776,...,22.995046,17.665243,19.659806,20.755871,20.048621,19.850872,48.472762,0.652898,0.612346,Q1


---

# load wav path

In [2]:
df = pd.read_csv('../csvs/new_dataset.csv', encoding='utf-8-sig')

In [3]:
from pathlib import Path
prefix = '../downloads/'

path0 = [prefix + v for v in df[df['submatch'] == 0]['video'].tolist()]
path1 = [prefix + v for v in df[df['submatch'] == 1]['video'].tolist()]
path2 = [prefix + v for v in df[df['submatch'] == 2]['video'].tolist()]
path3 = [prefix + v for v in df[df['submatch'] == 3]['video'].tolist()]
path1

['../downloads/Dear Name (이름에게).wav',
 "../downloads/너라서 (Because it's you).wav",
 '../downloads/Dear..wav',
 '../downloads/고쳐주세요.wav',
 '../downloads/처음부터 너와 나.wav',
 '../downloads/새벽한시 (1AM).wav',
 '../downloads/여수 밤바다.wav',
 '../downloads/봄날.wav',
 '../downloads/Beautiful Moment (내 생에 아름다운).wav',
 '../downloads/I need you (니가 필요해 (I NEED YOU)).wav',
 '../downloads/[도깨비 OST Part 9] 에일리 (Ailee) - 첫눈처럼 너에게 가겠다 (I will go to you like the first snow) (Official Audio).wav',
 '../downloads/Heaven (HEAVEN).wav',
 "../downloads/BEAST - '괜찮겠니 (Will you be alright_)' (Snippet).wav",
 '../downloads/SISTAR - LONELY Color Coded Lyrics [Han_Rom_Eng].wav',
 '../downloads/4.처음엔 사랑이란게 - Love, At First.wav']

In [4]:
pos = Path('../downloads')
pos_wav_paths = sorted(pos.glob('*.wav'))
len(pos_wav_paths)

123

In [ ]:
pos = Path('downloads/2')
pos_wav_paths = sorted(pos.glob('*.wav'))
neg = Path('downloads/3')
neg_wav_paths = sorted(neg.glob('*.wav'))

In [3]:
len(pos_wav_paths), len(neg_wav_paths)

(30, 31)

# create columns

In [5]:
df['tempo_transition_complexity'] = 0
df['active_bpm_variance'] = 0
df['pitch_high'] = 0
df['pitch_low'] = 0
df['pitch_range'] = 0
df['rms_variation'] = 0
df['repeat_area_size'] = 0
df['repetition_ratio'] = 0

# tempo transition complexity & active bpm number variance

In [12]:
def get_onset_tempogram_tempo(audio_path):
  # Load audio
  # y, sr = librosa.load(audio_path, duration=30)
  y, sr = librosa.load(audio_path)
  hop_length = 512

  # Onset strength envelope
  oenv = librosa.onset.onset_strength(y=y, sr=sr, hop_length=hop_length)

  # Tempogram (local autocorrelation)
  tempogram = librosa.feature.tempogram(onset_envelope=oenv, sr=sr, hop_length=hop_length)

  return tempogram

def get_dominant_tempo(tempogram, threshold=0.5):

  active_bpm_per_frame = np.sum(tempogram >= threshold, axis=0)
  active_bpm_variance = np.var(active_bpm_per_frame)

  prev_indices = None
  change_counts = 0
  for t in range(tempogram.shape[1]):
      indices = set(np.where(tempogram[:, t] >= threshold)[0])
      if prev_indices is not None:
          change = len(indices.symmetric_difference(prev_indices))
          change_counts += change
      prev_indices = indices
  tempo_transition_complexity = change_counts / tempogram.shape[1]

  return tempo_transition_complexity, active_bpm_variance

In [7]:
from tqdm.auto import tqdm
for wav_path in tqdm(pos_wav_paths):
    video_name = wav_path.stem
    tempogram = get_onset_tempogram_tempo(wav_path)
    tempo_transition_complexity, active_bpm_variance = get_dominant_tempo(tempogram)
    df.loc[df['video'] == video_name, 'tempo_transition_complexity'] = tempo_transition_complexity
    df.loc[df['video'] == video_name, 'active_bpm_variance'] = active_bpm_variance
    
for wav_path in tqdm(neg_wav_paths):
    video_name = wav_path.stem
    tempogram = get_onset_tempogram_tempo(wav_path)
    tempo_transition_complexity, active_bpm_variance = get_dominant_tempo(tempogram)
    df.loc[df['video'] == video_name, 'tempo_transition_complexity'] = tempo_transition_complexity
    df.loc[df['video'] == video_name, 'active_bpm_variance'] = active_bpm_variance
    

/opt/anaconda3/envs/mir/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/30 [00:00<?, ?it/s]

/var/folders/8s/9mw0byfj2yjdqnsnrkq34mkc0000gn/T/ipykernel_1315/1675278397.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.101242041540549' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['video'] == video_name, 'tempo_transition_complexity'] = tempo_transition_complexity
/var/folders/8s/9mw0byfj2yjdqnsnrkq34mkc0000gn/T/ipykernel_1315/1675278397.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '86.40402966762194' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['video'] == video_name, 'active_bpm_variance'] = active_bpm_variance
100%|██████████| 31/31 [00:16<00:00,  1.89it/s]


# tempo variance

In [ ]:
def check_tempo_variance(audio_path):
    y, sr = librosa.load(audio_path)
    hop_length = 512

    onset_strength_seq = librosa.onset.onset_strength(y=y, sr=sr, hop_length=hop_length) # 각 타임 프레임 시작점의 강도
    tempogram = librosa.feature.tempogram(onset_envelope=onset_strength_seq, sr=sr, hop_length=hop_length) # (num_bins, num_frames)

    tempos = librosa.tempo_frequencies(tempogram.shape[0], sr=sr, hop_length=hop_length) # tempo bin index → bpm
    dominant_indices = np.argmax(tempogram[1:], axis=0) + 1
    dominant_tempos = tempos[dominant_indices]

    dominant_tempos = dominant_tempos[(~np.isnan(dominant_tempos)) & (~np.isinf(dominant_tempos))]
    dominant_tempos = np.log2(dominant_tempos + 1e-6)

    if len(dominant_tempos) == 0:
        return 0.0

    tempo_var = np.var(dominant_tempos)

    return tempo_var

In [ ]:
for wav_path in tqdm(pos_wav_paths):
    video_name = wav_path.stem
    tempo_var = check_tempo_variance(wav_path)
    df.loc[df['video'] == video_name, 'tempo_variance'] = tempo_var

for wav_path in tqdm(neg_wav_paths):
    video_name = wav_path.stem
    tempo_var = check_tempo_variance(wav_path)
    df.loc[df['video'] == video_name, 'tempo_variance'] = tempo_var

100%|██████████| 31/31 [00:15<00:00,  2.03it/s]


# pitch range

In [15]:
def get_pitch_range(audio_path):
  # y, sr = librosa.load(audio_path, duration=30)
  y, sr = librosa.load(audio_path)

  f0, _, _ = librosa.pyin(
      y,
      fmin=librosa.note_to_hz('C2'),
      fmax=librosa.note_to_hz('C7'),
      sr=sr
  )

  # NaN 제거
  f0 = f0[~np.isnan(f0)]

  # IQR 기반 이상치 제거
  q1 = np.percentile(f0, 25)
  q3 = np.percentile(f0, 75)
  iqr = q3 - q1
  lower_bound = q1 - 1.5 * iqr
  upper_bound = q3 + 1.5 * iqr
  f0_filtered = f0[(f0 >= lower_bound) & (f0 <= upper_bound)]
  pitch_range = np.max(f0_filtered) - np.min(f0_filtered)

  return np.max(f0_filtered), np.min(f0_filtered), pitch_range

In [16]:
for wav_path in tqdm(neg_wav_paths):
    video_name = wav_path.stem
    pitch_high, pitch_low, pitch_range = get_pitch_range(wav_path)
    df.loc[df['video'] == video_name, 'pitch_high'] = pitch_high
    df.loc[df['video'] == video_name, 'pitch_low'] = pitch_low
    df.loc[df['video'] == video_name, 'pitch_range'] = pitch_range

for wav_path in tqdm(pos_wav_paths):
    video_name = wav_path.stem
    pitch_high, pitch_low, pitch_range = get_pitch_range(wav_path)
    df.loc[df['video'] == video_name, 'pitch_high'] = pitch_high
    df.loc[df['video'] == video_name, 'pitch_low'] = pitch_low
    df.loc[df['video'] == video_name, 'pitch_range'] = pitch_range

  0%|          | 0/31 [00:00<?, ?it/s]/var/folders/8s/9mw0byfj2yjdqnsnrkq34mkc0000gn/T/ipykernel_1315/3072824108.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '425.01198472693204' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['video'] == video_name, 'pitch_high'] = pitch_high
/var/folders/8s/9mw0byfj2yjdqnsnrkq34mkc0000gn/T/ipykernel_1315/3072824108.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '65.40639132514966' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['video'] == video_name, 'pitch_low'] = pitch_low
/var/folders/8s/9mw0byfj2yjdqnsnrkq34mkc0000gn/T/ipykernel_1315/3072824108.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Va

# repetition_ratio

In [ ]:
def check_repetition_ratio(audio_path, threshold=0.8, min_separation_sec=1.0, hop_length=512):
    y, sr = librosa.load(audio_path)
    
    chroma = librosa.feature.chroma_cqt(y=y, sr=sr, hop_length=hop_length)  # shape: (12, T)
    similarity_matrix = cosine_similarity(chroma.T)  # shape: (T, T)
    T = similarity_matrix.shape[0]

    # Minimum diagonal offset (exclude self or very nearby frames)
    min_diag_offset = int(min_separation_sec * sr / hop_length)

    time_idx = np.arange(T)
    frame_diff = np.abs(time_idx[:, None] - time_idx[None, :])

    # Valid repeat: high similarity + not too close in time
    is_valid_repeat = (similarity_matrix > threshold) & (frame_diff >= min_diag_offset)

    total_possible = np.sum(frame_diff >= min_diag_offset)
    repeated_area = np.sum(is_valid_repeat)
    repetition_ratio_value = repeated_area / total_possible if total_possible > 0 else 0

    return repeated_area, repetition_ratio_value

In [11]:
for wav_path in tqdm(pos_wav_paths):
    video_name = wav_path.stem
    repeated_area, repetition_ratio = check_repetition_ratio(wav_path)
    df.loc[df['video'] == video_name, 'repeat_area_size'] = repeated_area
    df.loc[df['video'] == video_name, 'repetition_ratio'] = repetition_ratio

for wav_path in tqdm(neg_wav_paths):
    video_name = wav_path.stem
    repeated_area, repetition_ratio = check_repetition_ratio(wav_path)
    df.loc[df['video'] == video_name, 'repeat_area_size'] = repeated_area
    df.loc[df['video'] == video_name, 'repetition_ratio'] = repetition_ratio

  0%|          | 0/30 [00:00<?, ?it/s]/var/folders/8s/9mw0byfj2yjdqnsnrkq34mkc0000gn/T/ipykernel_1315/3115970053.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.6490615794500998' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['video'] == video_name, 'repetition_ratio'] = repetition_ratio
100%|██████████| 31/31 [00:48<00:00,  1.57s/it]


# mean self-similarity matrix

In [ ]:
from skimage.transform import resize
import os
import numpy as np
import librosa
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

def check_repetition_ratio(audio_path, target_size=128):
    y, sr = librosa.load(audio_path)

    chroma = librosa.feature.chroma_cqt(y=y, sr=sr) # (12, T)
    similarity_matrix = cosine_similarity(chroma.T) # 각 타임 프레임 간 유사도 계산 (T, T)

    T = similarity_matrix.shape[0]

    repeated_area = np.sum(similarity_matrix)
    repetition_ratio_value = repeated_area / (T * T)

    resized_mask = resize(similarity_matrix.astype(float), (target_size, target_size),
                          mode='reflect', anti_aliasing=True, preserve_range=True)
    repetition_ratio_value = np.mean(resized_mask > 0.5)

    return repetition_ratio_value, resized_mask, similarity_matrix

def plot_silimilarity_matrix(similarity_matrix, save_path=None):
    plt.figure(figsize=(6, 5))
    plt.imshow(similarity_matrix, origin='lower', aspect='auto', cmap='magma', interpolation='nearest')
    plt.title("Mean Self-Similarity Matrix")
    plt.colorbar(label='Repetition Presence Probability')
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300)
        print(f"Saved visualization to: {save_path}")
    
    plt.close()


download_dir = "../downloads/1"
results = []
resized_masks = []

count = 0
max_files = 10
target_size = 128

for fname in tqdm(os.listdir(download_dir)):
    if fname.endswith(".wav"):
        path = os.path.join(download_dir, fname)
        rep_ratio, mask, sim_matrix = check_repetition_ratio(path, target_size)
        resized_masks.append(mask)

# 평균 마스크 시각화
mean_matrix = np.mean(resized_masks, axis=0)
plot_silimilarity_matrix(mean_matrix, '../imgs/1.png')

In [ ]:
download_dir = "../downloads/2"
results = []
resized_masks = []

count = 0
max_files = 10
target_size = 128

for fname in tqdm(os.listdir(download_dir)):
    if fname.endswith(".wav"):
        path = os.path.join(download_dir, fname)
        rep_ratio, mask, sim_matrix = check_repetition_ratio(path, target_size)
        resized_masks.append(mask)

# 평균 마스크 시각화
mean_matrix = np.mean(resized_masks, axis=0)
plot_silimilarity_matrix(mean_matrix, '../imgs/2.png')

In [ ]:
download_dir = "../downloads/3"
results = []
resized_masks = []

count = 0
max_files = 10
target_size = 128

for fname in tqdm(os.listdir(download_dir)):
    if fname.endswith(".wav"):
        path = os.path.join(download_dir, fname)
        rep_ratio, mask, sim_matrix = check_repetition_ratio(path, target_size)
        resized_masks.append(mask)

# 평균 마스크 시각화
mean_matrix = np.mean(resized_masks, axis=0)
plot_silimilarity_matrix(mean_matrix, '../imgs/3.png')

In [ ]:
download_dir = "../downloads/0"
results = []
resized_masks = []

count = 0
max_files = 10
target_size = 128

for fname in tqdm(os.listdir(download_dir)):
    if fname.endswith(".wav"):
        path = os.path.join(download_dir, fname)
        rep_ratio, mask, sim_matrix = check_repetition_ratio(path, target_size)
        resized_masks.append(mask)

# 평균 마스크 시각화
mean_matrix = np.mean(resized_masks, axis=0)
plot_silimilarity_matrix(mean_matrix, '../imgs/0.png')

# rms variation

In [8]:
def check_rms_variation(audio_path):
    # y, sr = librosa.load(audio_path, duration=30)
    y, sr = librosa.load(audio_path)

    rms = librosa.feature.rms(y=y).flatten()

    rms = rms[~np.isnan(rms) & ~np.isinf(rms)]

    if len(rms) == 0:
        return 0.0

    rms_var = np.var(rms)
    return rms_var

In [9]:
for wav_path in tqdm(neg_wav_paths):
    video_name = wav_path.stem
    rms_variation = check_rms_variation(wav_path)
    df.loc[df['video'] == video_name, 'rms_variation'] = rms_variation

for wav_path in tqdm(pos_wav_paths):
    video_name = wav_path.stem
    rms_variation = check_rms_variation(wav_path)
    df.loc[df['video'] == video_name, 'rms_variation'] = rms_variation

  0%|          | 0/31 [00:00<?, ?it/s]/var/folders/8s/9mw0byfj2yjdqnsnrkq34mkc0000gn/T/ipykernel_1315/3128366921.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.007276649121195078' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['video'] == video_name, 'rms_variation'] = rms_variation
100%|██████████| 30/30 [00:09<00:00,  3.24it/s]


---

In [4]:
from skimage.transform import resize
import os
import numpy as np
import librosa
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

def check_repetition_ratio(audio_path, target_size=128):
    y, sr = librosa.load(audio_path)

    chroma = librosa.feature.chroma_cqt(y=y, sr=sr) # (12, T)
    similarity_matrix = cosine_similarity(chroma.T) # 각 타임 프레임 간 유사도 계산 (T, T)

    T = similarity_matrix.shape[0]

    repeated_area = np.sum(similarity_matrix)
    repetition_ratio_value = repeated_area / (T * T)

    resized_mask = resize(similarity_matrix.astype(float), (target_size, target_size),
                          mode='reflect', anti_aliasing=True, preserve_range=True)
    repetition_ratio_value = np.mean(resized_mask > 0.5)

    return repetition_ratio_value, resized_mask, similarity_matrix

def plot_silimilarity_matrix(similarity_matrix, save_path=None):
    plt.figure(figsize=(6, 5))
    plt.imshow(similarity_matrix, origin='lower', aspect='auto', cmap='magma', interpolation='nearest')
    plt.title("Mean Self-Similarity Matrix")
    plt.colorbar(label='Repetition Presence Probability')
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300)
    
    plt.close()

/opt/anaconda3/envs/mir/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
results = []
resized_masks = []
target_size = 128

for fname in tqdm(path0):
    rep_ratio, mask, sim_matrix = check_repetition_ratio(fname, target_size)
    resized_masks.append(mask)
    img = fname.split('/')[-1].split('.wav')[0]
    plot_silimilarity_matrix(sim_matrix, f"../imgs/0/{img}.png")

# 평균 마스크 시각화
mean_matrix = np.mean(resized_masks, axis=0)
plot_silimilarity_matrix(mean_matrix, '../imgs/0_new.png')

  0%|          | 0/31 [00:00<?, ?it/s]

100%|██████████| 31/31 [14:35<00:00, 28.25s/it]


In [6]:
results = []
resized_masks = []
target_size = 128

for fname in tqdm(path1):
    rep_ratio, mask, sim_matrix = check_repetition_ratio(fname, target_size)
    resized_masks.append(mask)
    img = fname.split('/')[-1].split('.wav')[0]
    plot_silimilarity_matrix(sim_matrix, f"../imgs/1/{img}.png")

# 평균 마스크 시각화
mean_matrix = np.mean(resized_masks, axis=0)
plot_silimilarity_matrix(mean_matrix, '../imgs/1_new.png')

100%|██████████| 15/15 [08:31<00:00, 34.09s/it]


In [7]:
results = []
resized_masks = []
target_size = 128

for fname in tqdm(path2):
    rep_ratio, mask, sim_matrix = check_repetition_ratio(fname, target_size)
    resized_masks.append(mask)
    img = fname.split('/')[-1].split('.wav')[0]
    plot_silimilarity_matrix(sim_matrix, f"../imgs/2/{img}.png")

# 평균 마스크 시각화
mean_matrix = np.mean(resized_masks, axis=0)
plot_silimilarity_matrix(mean_matrix, '../imgs/2_new.png')

100%|██████████| 45/45 [22:19<00:00, 29.77s/it]


In [8]:
results = []
resized_masks = []
target_size = 128

for fname in tqdm(path3):
    rep_ratio, mask, sim_matrix = check_repetition_ratio(fname, target_size)
    resized_masks.append(mask)
    img = fname.split('/')[-1].split('.wav')[0]
    plot_silimilarity_matrix(sim_matrix, f"../imgs/3/{img}.png")

# 평균 마스크 시각화
mean_matrix = np.mean(resized_masks, axis=0)
plot_silimilarity_matrix(mean_matrix, '../imgs/3_new.png')

100%|██████████| 27/27 [14:57<00:00, 33.23s/it]
